# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook
%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues

apk_dir = "../../../Downloads/"

# APK Files to load
file_paths = (
    "com.snapchat.android_11.5.0.69-2118_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.snapchat.android_11.6.1.66-2125_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
)


file_paths = (
    "com.instagram.android_169.1.0.29.135-262886990_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    "com.instagram.android_169.1.0.29.135-262886984_minAPI21(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)
file_paths = tuple(map(lambda apk_name: apk_dir + apk_name, file_paths))

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_169.1.0.29.135-262886990_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_169.1.0.29.135-262886984_minAPI21(armeabi-v7a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Fields

Defining Elements that GraphGuard should try to match (obviously requires full class names).

GraphGuard has full Akrolyb support. Read about it at [Akrolyb Interoptability](#Interoptability-with-Akrolyb). Using this is optional. In case you don't want to use this functionality, do not run the few last cells that are related to Akrolyb Support and Automation


* For using Akrolyb Support:
    * Use CodeGen in Akrolyb to generate a Python-compatible tuple of declarations that you can copy paste into `named_c_decs`, `named_f_decs` and `named_f_decs`.
    * Specify the correct file for auto replacing values, the "declarations files".
    * Types of the variables:
        * `named_c_decs` in form of a `Dict[str, str]`
        * `named_m_decs` in form of a `Dict[str, MethodDec]`
        * `named_f_decs` in form of a `Dict[str, FieldDec]`
    * GraphGuard extracts the declarations and uses the given names for Auto-Updates.


* For not using Akrolyb, and just simply try and match the values (without Auto-Replacing):
    * Leave named_c_decs, named_m_decs, named_f_decs empty.
    * Assign the declarations directly to c_decs, m_decs, f_decs further below by uncommenting the existing cell.
    * Types of the variables:
        * `c_decs` in form of a `Tuple[str]`
        * `m_decs` in form of a `Tuple[MethodDec]`
        * `f_decs` in form of a `Tuple[FieldDec]`

Note that classes of methods and fields are automatically merged with `c_decs`.

In [5]:
named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPEXPERIMENTHELPER': 'X.1YO',
    'CLIPITEM': 'X.8pU',
    'COMMENT_LONGCLICK_HANDLER': 'X.4Vw',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3Gy',
    'DM_HOLDER': 'X.5kO',
    'DM_REPLY': 'X.4aV',
    'DM_VOICEMESSAGE': 'X.5RF',
    'DMVIDEOPLAYER': 'X.6QV',
    'FEEDITEM': 'X.28M',
    'FRAGMENT': 'androidx.fragment.app.Fragment',
    'HEADERGENERATOR': 'X.0Rs',
    'HTTPREQUESTTASKFACTORY': 'X.2I7',
    'INAPP_NOTIFICATION': 'X.5CQ',
    'IGTV_VIEWER_FRAGMENT': 'X.9oB',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.0xP',
    'JACKSON_CREATOR': 'X.0k0',
    'LIVE_POST': 'X.2NY',
    'MAINFEED': 'X.1J1',
    'MEDIAITEM': 'X.1Ov',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1ho',
    'NOTIFICATION_HANDLER': 'X.2hn',
    'PHOTOTIMERCONTROLLER': 'X.3e0',
    'POSTDETAIL': 'X.1XF',
    'POSTGESTURE_SIMPLEPOST': 'X.2A2',
    'POSTGESTURE_CAROUSELIMAGE': 'X.2Bi',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2St',
    'POSTVIEW': 'X.1XK',
    'PROFILE': 'X.2OY',
    'PROFILEDETAIL': 'X.3os',
    'PROFILEPIC_LONGCLICK': 'X.3ot',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2To',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.2QR',
    'RESPONSE_ASYNC_HANDLER': 'X.2Pd',
    'NOTIFICATION_RUNNABLE': 'X.56n',
    'SERIALSCHEDULER': 'X.2SP',
    'SESSION': 'X.0Mp',
    'SIMPLE_EXECUTOR': 'X.4kG',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1RF',
    'VIDEOSETTINGSMANAGER': 'X.1bM',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPEXPERIMENTHELPER_ISREDESIGN': MethodDec('X.1YO', 'A09', 'X.0Mp'),
    'CLIPITEM_UPDATEMEDIA': MethodDec('X.8pU', 'A03', 'X.8pU', 'X.8oT', 'X.2BH', 'int', 'java.lang.String'),
    'COMMENT_ONLONGCLICK': MethodDec('X.4Vw', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.17W', 'androidx.fragment.app.FragmentActivity', 'X.0Mp', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mp'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0Mp'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3Gy', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.5kO', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.4aV', 'A0K', 'X.4aV', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.4aV', 'A0G', 'X.4aV'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.6QV', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.28M', 'A00', 'X.0kA'),
    'FRAGMENT_GETACTIVITY': MethodDec('androidx.fragment.app.Fragment', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0Rs', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.2I7', 'A03', 'java.lang.String', 'X.0Mp'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.5CQ', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.9oB', 'A00', 'X.9oB'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.9oB', 'A0C', 'X.9oB'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.0xP', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0k0', 'A03', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0k0', '<init>', 'X.GBP'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.2NY', 'A00', 'X.0kc', 'X.2NZ'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AHw'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1J1', 'configureActionBar', 'X.1EB'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1Ov', 'A00', 'X.0kA', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0kc', 'X.1Ov'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1ho', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2hn', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.3e0', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1XF', 'A08', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.2A2', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.2Bi', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2St', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1XK', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3os', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.2OY', 'A04', 'X.0Mp', 'X.0kV'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3ot', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2To', 'A0U', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'BVz', 'java.lang.Object'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0P', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0U', 'com.instagram.reels.fragment.ReelViewerFragment', 'boolean'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0Y', 'X.1k6', 'X.1oK'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.2QR', 'A00', 'X.2QQ', 'X.2Pc'),
    'RESPONSE_ASYNC_HANDLER_ONRESPONSE': MethodDec('X.2Pd', 'A02', 'X.2Pc', 'X.1xM'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.56n', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.2SP', 'A02', 'X.0nA'),
    'SESSION_GETTOKEN': MethodDec('X.0Mp', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4kG', 'onFail', 'X.2Hq'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4kG', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4kG', '<init>', 'X.0Mp'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1RF', 'A03', 'X.0kc', 'X.0kU'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1bM', 'A00', 'X.0Mp'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1EA', 'A09'),
    'CLIPSITEM_SESSION': FieldDec('X.8pU', 'A03'),
    'CLIPSVIEW_PARENTVIEW': FieldDec('X.8oT', 'A00'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.2BH', 'A01'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.4Vw', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1Rd', 'A0a'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3Gy', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.6PV', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.6PY', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.5RF', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.28M', 'A0J'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0sR', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.9oB', 'A0X'),
    'MAINFEED_SESSION': FieldDec('X.1J1', 'A0c'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1ON', 'A07'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1oK', 'A0D'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1oK', 'A0C'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.2QT', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.2NX', 'A01'),
    'NATIVEINPUTSTREAMASYNCHOLDER_NATIVEINPUTSTREAM': FieldDec('X.18k', 'A03'),
    'NATIVEINPUTSTREAMASYNCHOLDER_URI': FieldDec('X.18k', 'A04'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.5CQ', 'A00'), FieldDec('X.56n', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.5CQ', 'A01'), FieldDec('X.56n', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1nX', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1na', 'A01'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3kj', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3kj', 'A05'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3kj', 'A06'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3kj', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3kl', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.2NV', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.3YX', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1P'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.2LW', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1oK', 'A0H')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

In [9]:
# Without using Akrolyb's Auto-Replacing Values:

# c_decs = tuple()
# m_decs = tuple()
# f_decs = tuple()

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [10]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [11]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [12]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [13]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	X.1ho#A01() -> X.1hr#A01()
+ Found single candidate for Method. Considering it a match 
	com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController) -> com.instagram.direct.visual.DirectVisualMessageViewerController#A04(com.instagram.direct.visual.DirectVisualMessageViewerController)
+ Found single candidate for Method. Considering it a match 
	X.0Rs#A00(android.content.Context) -> X.0Rs#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	X.1J1#configureActionBar(X.1EB) -> X.1J4#configureActionBar(X.1EE)
+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0Y(X.1k6, X.1oK) -> com.instagram.reels.fragment.ReelViewerFragment#A0Y(X.1k9, X.1oN)
+ Found single candidate for Method. Considering it a match 
	com.instagram.mainactivity.MainActivity#onDestroy() -> co

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [14]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/0Mp; -> LX/0Mp;
+ Found single candidate for Class. Considering it a match! 
	LX/0sR; -> LX/0sU;
+ Found single candidate for Class. Considering it a match! 
	LX/0xP; -> LX/0xS;
+ Found single candidate for Class. Considering it a match! 
	Landroidx/fragment/app/Fragment; -> Landroidx/fragment/app/Fragment;
+ Found single candidate for Class. Considering it a match! 
	LX/18k; -> LX/18n;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer; -> Lcom/instagram/ui/swipenavigation/SwipeNavigationContainer;
+ Found single candidate for Class. Considering it a match! 
	LX/1EA; -> LX/1ED;
+ Found single candidate for Class. Considering it a match! 
	LX/1Ov; -> LX/1Oy;
+ Found single candidate for Class. Considering it a match! 
	LX/1RF; -> LX/1RI;
+ Found single candidate for Class. Considering it a match! 
	LX/1Rd; -> LX/1Rg;
+ Found single candidate for Class. 

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [15]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/4Vw; -> LX/4W1;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3Gy; -> LX/3H3;
+ Found single candidate for Class. Considering it a match! 
	LX/5kO; -> LX/5kP;
+ Found single candidate for Class. Considering it a match! 
	LX/6QV; -> LX/6QE;
+ Found single candidate for Class. Considering it a match! 
	LX/0k0; -> LX/0k3;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/3e0; -> LX/3e5;
+ Found single candidate for Class. Considering it a match! 
	LX/2A2; -> LX/2A5;
* Found multiple candidates for Class LX/2Bi;
* Found multiple candidates for Class LX/2S

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [16]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.2Bi#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2St#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	X.8pU#A03(X.8pU, X.8oT, X.2BH, int, java.lang.String) -> X.8pK#A03(X.8pK, X.8oA, X.2BK, int, java.lang.String)
+ Found single candidate for Method. Considering it a match 
	X.4Vw#onLongPress(android.view.MotionEvent) -> X.4W1#onLongPress(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.17W, androidx.fragment.app.FragmentActivity, X.0Mp, android.os.Bundle) -> com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#loadAndLaunchDeveloperOptions(android.content.Context, X.17Z, androidx.fragment.app.FragmentActivity, X.0Mp, android.os.Bundle)
* Found multiple candidates for Method com.instagram.debug.devoptions

+ Found single candidate for Method. Considering it a match 
	androidx.fragment.app.Fragment#getActivity() -> androidx.fragment.app.Fragment#getActivity()
+ Found single candidate for Method. Considering it a match 
	X.3os#A02(X.3kj, android.content.Context, X.0Mp, X.0kU, X.0Sx, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1Ov, java.lang.Integer, boolean, boolean, X.1br) -> X.3ox#A02(X.3ko, android.content.Context, X.0Mp, X.0kX, X.0Sx, com.instagram.profile.fragment.UserDetailDelegate, boolean, X.1Oy, java.lang.Integer, boolean, boolean, X.1bu)
Could resolve 0 new Classes, 2 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [17]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0kU; -> LX/0kX;
+ Found single candidate for Class. Considering it a match! 
	LX/1SS; -> LX/1SV;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/2NZ; -> LX/2Nc;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/2D5; -> LX/2D8;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/gesture/GestureManagerFrameLayout; -> Lcom/instagram/ui/gesture/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a 

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [18]:
print(len(accumulator.matching_cs), "/", len(c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

78 / 153 Classes were matched
58 / 62 Methods were matched
41 / 43 Fields were matched

Matching Classes:
• X.1ho -> X.1hr
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.0Rs -> X.0Rs
• X.1J1 -> X.1J4
• com.instagram.reels.fragment.ReelViewerFragment -> com.instagram.reels.fragment.ReelViewerFragment
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• X.1YO -> X.1YR
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.4aV -> X.4aa
• X.2QR -> X.2QU
• X.1bM -> X.1bP
• X.2Pd -> X.2Pg
• X.2To -> X.2Tr
• X.4kG -> X.4kb
• X.5CQ -> X.Fkk
• X.56n -> X.Fkl
• X.0Mp -> X.0Mp
• X.0sR -> X.0sU
• X.0xP -> X.0xS
• androidx.fragment.app.Fragment -> androidx.fragment.app.Fragment
• X.18k -> X.18n
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1EA 

• com.facebook.proxygen.JniHandler#sendRequestWithBodyAndEom(org.apache.http.client.methods.HttpUriRequest, byte[], int, int) -> com.facebook.proxygen.JniHandler#sendRequestWithBodyAndEom(org.apache.http.client.methods.HttpUriRequest, byte[], int, int)
• com.instagram.reels.fragment.ReelViewerFragment#BVz(java.lang.Object) -> com.instagram.reels.fragment.ReelViewerFragment#BW4(java.lang.Object)
• com.instagram.reels.fragment.ReelViewerFragment#A0P(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String) -> com.instagram.reels.fragment.ReelViewerFragment#A0P(com.instagram.reels.fragment.ReelViewerFragment, java.lang.String)
• com.instagram.reels.fragment.ReelViewerFragment#A0U(com.instagram.reels.fragment.ReelViewerFragment, boolean) -> com.instagram.reels.fragment.ReelViewerFragment#A0U(com.instagram.reels.fragment.ReelViewerFragment, boolean)
• X.2SP#A02(X.0nA) -> X.2SS#A02(X.0nD)
• X.0Mp#getToken() -> X.0Mp#getToken()
• X.4kG#<init>(X.0Mp) -> X.4kb#<init>(X.0Mp)
• com.insta

In [19]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(c_decs, r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    MethodDec('X.1YR', 'A09', 'X.0Mp'),
    MethodDec('X.8pK', 'A03', 'X.8pK', 'X.8oA', 'X.2BK', 'int', 'java.lang.String'),
    MethodDec('X.4W1', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.17Z', 'androidx.fragment.app.FragmentActivity', 'X.0Mp', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mp),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mp),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.Dire

    'androidx.fragment.app.Fragment'
    'X.0Rs'
    'X.2IA'
    'X.Fkk'
    'X.9om'
    'com.instagram.realtimeclient.RealtimeClientManager'
    'X.0xS'
    'X.0k3'
    'X.2Nb'
    'X.1J4'
    'X.1Oy'
    'com.instagram.feed.media.Media__JsonHelper'
    'X.1hr'
    'X.2hq'
    'X.3e5'
    'X.1XI'
    'X.2A5'
    # No Match Found for Class 'X.2Bi'
    # No Match Found for Class 'X.2St'
    'X.1XN'
    'X.2Ob'
    'X.3ox'
    'X.3oy'
    'com.facebook.proxygen.JniHandler'
    'X.2Tr'
    'com.instagram.reels.fragment.ReelViewerFragment'
    'X.2QU'
    'X.2Pg'
    'X.Fkl'
    'X.2SS'
    'X.0Mp'
    'X.4kb'
    'com.instagram.ui.swipenavigation.SwipeNavigationContainer'
    'X.1RI'
    'X.1bP'
    'com.instagram.ui.widget.volume.VolumeIndicator',
)
f_decs = (
    FieldDec('X.1ED', 'A09'),
    FieldDec('X.8pK', 'A03'),
    FieldDec('X.8oA', 'A00'),
    FieldDec('X.2BK', 'A01'),
    FieldDec('X.4W1', 'A04'),
    FieldDec('X.1Rg', 'A0a'),
    FieldDec('com.instagram.direct.visual.DirectVis

# Warnings

Shows some warnings that result in common bugs, so that the Hooks (or other kind of modifications applied) must be updated logic-wise. 

Currently supports: 
* Change in number of parameters 

In [20]:
for m_old, m_new in accumulator.matching_ms.items():
    def get_number_of_params(m):
        return len(tuple(get_pretty_params(str(m.get_descriptor()))))
    
    if get_number_of_params(m_old) != get_number_of_params(m_new):
        print("Warning: Number of parameters changed for:", pretty_format_ma(m_old), "-", pretty_format_ma(m_new))

# Interoptability with Akrolyb

GraphGuard was build with Akrolyb in mind and has full Akrolyb support, meaning that it is not only capable of automatically matching the given declarations, but also automatically editing the "declaration file" to update matched declarations and mark unmatched items with a `/* TODO */` comment.

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. See [this](#List-of-Classes,-Methods-and-Fields) to learn how to use GraphGuard's Akrolyb Support.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```

In [21]:
file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/"
c_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = file_dir + "packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"


file_dir = "/home/jaqxues/CodeProjects/IdeaProjects/Instaprefs/"
c_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = file_dir + "packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"

In [22]:
%autoreload
c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mp)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0Mp)
No matching Method found for X.2Bi#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2St#onDoubleTap(android.view.MotionEvent)
No matching Field found for X.1nX#A06
No matching Field found for X.1na#A01


In [23]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPEXPERIMENTHELPER = ClassDec("X.1YR")

    override val CLIPITEM = ClassDec("X.8pK")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.4W1")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.3H3")

    override val DM_HOLDER = ClassDec("X.5kP")

    override val DM_REPLY = ClassDec("X.4aa")

    override val DM_VOICEMESSAGE = ClassDec("X.5RJ")

    override val DMVIDEOPLAYER = ClassDec("X.6QE")

    override val FEEDITEM = ClassDec("X.28P")

  

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream
import java.net.URI

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1ED"])
    override val ACTIONBAR_BASEVIEW = VariableDec<ViewGroup>("A09")

    @FieldClass(["X.8pK"])
    override val CLIPSITEM_SESSION = VariableDec<Any>("A03")

    @FieldClass(["X.8oA"])
    override val CLIPSVIEW_PARENTVIEW = VariableDec<FrameLayout>("A00")

    @FieldClass(["X.2BK"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A01")

    @FieldClass(["X.4W1"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1Rg"])
    override val COMMENT_TEXT = Variabl